# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.1 MB/s 
     |████████████████████████████████| 2.8 MB 23.2 MB/s 
     |████████████████████████████████| 57 kB 7.8 MB/s 
     |████████████████████████████████| 806 kB 65.2 MB/s 
     |████████████████████████████████| 636 kB 78.6 MB/s 
     |████████████████████████████████| 133 kB 84.1 MB/s 
     |████████████████████████████████| 332 kB 61.3 MB/s 
     |████████████████████████████████| 829 kB 76.1 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 895 kB 71.3 MB/s 
     |████████████████████████████████| 3.3 MB 66.9 MB/s 
     |████████████████████████████████| 1.1 MB 69.7 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 271 kB 77.9 MB/s 
     |████████████████████████████████| 160 kB 61.7 MB/s 
     |████████████████████████████████| 192 kB 82.2 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
from ratsnlp.nlpbook.ner import NERDeployArguments

args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=2-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [4]:
import torch
from transformers import BertConfig, BertForTokenClassification

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForTokenClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 레이블 맵 작성

범주 인덱스를 범주명과 매칭하는 사전을 만듭니다.

In [6]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}

for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [7]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
                
    return {
        "sentence": sentence,
        "result": result,
    }

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [8]:
!mkdir /root/.ngrok2 && echo "authtoken: " > /root/.ngrok2/ngrok.yml

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.ner import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()